# Boolean Logic

In this notebook, we'll explore how to use boolean logic to combine multiple directives. The Elasticsearch Query Toolkit provides powerful boolean directives that allow you to create complex query conditions.

__NOTE__: This is a demo on how Boolean Directives work under the hood but they are __not to be used directly__. The correct way of using Boolean Directives is inside a `CustomMatchDirective` which will come in the upcoming tutorials.

## Introduction to Boolean Directives

Boolean directives allow you to combine multiple directives using logical operators. The toolkit provides two main boolean directives:

- **AndDirective**: Requires all child directives to match (logical AND)
- **OrDirective**: Requires at least one child directive to match (logical OR)

These directives can be nested to create complex boolean expressions.

## Setup

Let's import the necessary modules:

In [ ]:
import json
from elastictoolkit.queryutils.builder.directivevaluemapper import DirectiveValueMapper
from elastictoolkit.queryutils.builder.matchdirective import (
    ConstMatchDirective,
    TextMatchDirective,
    RangeMatchDirective,
    FieldExistsDirective
)
from elastictoolkit.queryutils.builder.booldirective import (
    AndDirective,
    OrDirective
)
from elastictoolkit.queryutils.consts import FieldMatchType, MatchMode

# Helper function to print queries as formatted JSON
def print_query(directive):
    query = directive.to_dsl()
    print(json.dumps(query.to_query(), indent=2))

## Basic Boolean Queries

Let's start with a simple example of using boolean directives to combine multiple match conditions:

In [ ]:
class ValueMapper(DirectiveValueMapper):
    pass

# Create match directives
category_directive = ConstMatchDirective(rule=FieldMatchType.ANY)
category_directive.set_field("category")
category_directive.set_values("electronics")
category_directive.set_match_params({})

in_stock_directive = ConstMatchDirective(rule=FieldMatchType.ANY)
in_stock_directive.set_field("in_stock")
in_stock_directive.set_values(True)
in_stock_directive.set_match_params({})

# Combine with AND
and_directive = AndDirective(category=category_directive, in_stock=in_stock_directive)
and_directive.set_directive_value_mapper(ValueMapper())
and_directive.set_match_params({})

print("AND Directive:")
print_query(and_directive)

In [ ]:
# Combine with OR
or_directive = OrDirective(category=category_directive, in_stock=in_stock_directive)
or_directive.set_directive_value_mapper(ValueMapper())
or_directive.set_match_params({})

print("OR Directive:")
print_query(or_directive)

In these examples:

- The `AndDirective` generates a query that requires both conditions to match (category must be "electronics" AND in_stock must be true)
- The `OrDirective` generates a query that requires at least one condition to match (category must be "electronics" OR in_stock must be true)

## Nested Boolean Logic

Boolean directives can be nested to create complex boolean expressions:

In [ ]:
# Create additional directives
price_directive = RangeMatchDirective()
price_directive.set_field("price")
price_directive.set_values(gte=50, lte=200)
price_directive.set_match_params({})

brand_directive = ConstMatchDirective(rule=FieldMatchType.ANY)
brand_directive.set_field("brand")
brand_directive.set_values(["Apple", "Samsung"])
brand_directive.set_match_params({})

# Create a nested boolean expression: (category AND in_stock) OR (price AND brand)
and_directive1 = AndDirective(category=category_directive, in_stock=in_stock_directive)
and_directive1.set_match_params({})
and_directive1.set_directive_value_mapper(ValueMapper())

and_directive2 = AndDirective(price=price_directive, brand=brand_directive)
and_directive2.set_match_params({})
and_directive2.set_directive_value_mapper(ValueMapper())

or_directive = OrDirective(and_directive1, and_directive2)
or_directive.set_match_params({})
or_directive.set_directive_value_mapper(ValueMapper())

print("Nested Boolean Expression:")
print_query(or_directive)

This example demonstrates how to create a complex boolean expression: (category AND in_stock) OR (price AND brand). The resulting query will match documents that either:

1. Have category="electronics" AND in_stock=true, OR
2. Have price between 50 and 200 AND brand is either "Apple" or "Samsung"

## Dynamic Value Resolution in Boolean Logic

Boolean directives can also work with dynamic values from match parameters:

In [ ]:
from elastictoolkit.queryutils.builder.directivevaluemapper import DirectiveValueMapper
from elastictoolkit.queryutils.types import FieldValue

# Define a value mapper
class ProductValueMapper(DirectiveValueMapper):
    category = FieldValue(
        fields=["category"],
        values_list=["match_params.category"]
    )
    price_range = FieldValue(
        fields=["price"],
        values_map={
            "gte": "match_params.min_price",
            "lte": "match_params.max_price"
        }
    )
    brand = FieldValue(
        fields=["brand"],
        values_list=["*match_params.brands"]
    )

# Create match parameters
match_params = {
    "category": "electronics",
    "min_price": 50,
    "max_price": 200,
    "brands": ["Apple", "Samsung"]
}

# Combine with AND | Field and Values will be mapped directly via `ProductValueMapper`
and_directive = AndDirective(
    category=ConstMatchDirective(rule=FieldMatchType.ANY), 
    price_range=RangeMatchDirective(), 
    brand=ConstMatchDirective(rule=FieldMatchType.ANY),
)
and_directive.set_match_params(match_params)
and_directive.set_directive_value_mapper(ProductValueMapper())

print("Dynamic Boolean Query:")
print_query(and_directive)

This example demonstrates how to create a boolean query with dynamic values from match parameters. The resulting query will match documents that have:

1. category="electronics" AND
2. price between 50 and 200 AND
3. brand is either "Apple" or "Samsung"

The values are resolved dynamically from the match parameters, making the query adaptable to different inputs.

This example demonstrates how to create conditional boolean logic based on the presence of certain parameters. The function `create_product_query` builds a query that always includes a category filter, but only includes price and brand filters if the corresponding parameters are present.

## Boolean Logic in Directive Engines

The directive engine itself generates a boolean query, the base boolean condition by default is `AND` but is configurable and can be changed to `OR`. For `NOT` queries always use `MatchMode.EXCLUDE` for `mode` parm in `MatchDirective`(s):

In [ ]:
from elastictoolkit.queryutils.builder.directiveengine import DirectiveEngine
from elastictoolkit.queryutils.consts import BaseMatchOp, MatchMode

class ProductSearchEngine(DirectiveEngine):
    # Define directives as class attributes
    category = ConstMatchDirective(rule=FieldMatchType.ANY)
    price_range = RangeMatchDirective()
    brand = ConstMatchDirective(rule=FieldMatchType.ANY, mode=MatchMode.EXCLUDE)
    
    class Config:
        value_mapper = ProductValueMapper()
        match_directive_config = {
            "base_match_op": BaseMatchOp.AND  # Default is AND
        }

match_params = {
    "category": "electronics",
    "min_price": 50,
    "max_price": 200,
    "brands": ["Apple", "Samsung"]
}

# Create an engine instance with match parameters
engine = ProductSearchEngine().set_match_params(match_params)

print("AND Engine:")
print_query(engine)

# Create an OR engine
class OrProductSearchEngine(ProductSearchEngine):
    class Config:
        value_mapper = ProductValueMapper()
        match_directive_config = {
            "base_match_op": BaseMatchOp.OR  # Change to OR
        }

# Create an engine instance with match parameters
or_engine = OrProductSearchEngine().set_match_params(match_params)

print("\nOR Engine:")
print_query(or_engine)

This example demonstrates how to use directive engines to handle boolean logic. The `ProductSearchEngine` combines directives using AND logic by default, while the `OrProductSearchEngine` combines them using OR logic. This approach is more declarative and maintainable than manually creating boolean directives.

## Summary

In this notebook, we've explored how to use boolean logic to combine multiple directives. We've covered:

- Basic boolean queries with AND and OR directives
- Nested boolean expressions
- Dynamic boolean logic with match parameters
- Boolean logic in directive engines

Boolean directives provide a powerful way to create complex query conditions by combining multiple directives using logical operators. When combined with directive engines, they offer a declarative, maintainable approach to building complex search queries.

In the next notebook, we'll explore how to create custom directives to encapsulate complex query patterns.